# Test notebook for Acceptance Test Campaign related to LSST Science Pipelines Release 20.0

This test will be executed on the LSST Science Platform Notebook Aspect, initialized with Science Pipelines release `r20-0-0`.

### Test case LVV-T1947: Verify implementation of measurements in catalogs from difference images
Verify that source measurements in catalogs are in flux units.

In [1]:
# Confirm that the version of the Science Pipelines is v20_0_0:
! echo $HOSTNAME
! eups list -s | grep lsst_distrib

nb-jcarlin-r20-0-0
lsst_distrib          20.0.0     	current v20_0_0 setup


In [2]:
import lsst.daf.persistence as dafPersist

In [3]:
import numpy as np

## Check a dataset with difference imaging

In [4]:
# Meredith pointed me to where she regularly re-runs HiTS data through diff imaging:
diffim_repo = '/project/mrawls/hits2015/rerun/cw_2020_08'

In [6]:
# This cell is a workaround for an issue discussed in #dm-butler on March 16, 2020. 
# The relevant Jira ticket is DM-21333.

#import lsst.obs.base as obs_base
#obs_base.FilterDefinitionCollection.reset() 

In [7]:
butler_dia = dafPersist.Butler(diffim_repo)

In [8]:
dataId_dia = {'visit': 411406, 'ccd': 15}

In [13]:
# To discover what dataset types are there:
#ddd = butler_dia.getDatasetTypes()
#for d in ddd:
#    if 'Src' in d:
#        print(d)

In [10]:
deepDiff_dia_src = butler_dia.get('deepDiff_diaSrc', dataId = dataId_dia)

In [11]:
sch_dia = deepDiff_dia_src.schema

In [12]:
dia_src_flag = []
for a in sch_dia:
    nnn = a.getField().getName()
    if 'instFlux' in nnn and 'flag' not in nnn:
        flux_units = a.getField().getUnits()
        dia_src_flag.append('count' in flux_units)
        print(f'{nnn:60}..... {flux_units:20}')

base_SdssShape_instFlux                                     ..... count               
base_SdssShape_instFluxErr                                  ..... count               
base_SdssShape_instFlux_xx_Cov                              ..... count*pixel^2       
base_SdssShape_instFlux_yy_Cov                              ..... count*pixel^2       
base_SdssShape_instFlux_xy_Cov                              ..... count*pixel^2       
base_CircularApertureFlux_3_0_instFlux                      ..... count               
base_CircularApertureFlux_3_0_instFluxErr                   ..... count               
base_CircularApertureFlux_4_5_instFlux                      ..... count               
base_CircularApertureFlux_4_5_instFluxErr                   ..... count               
base_CircularApertureFlux_6_0_instFlux                      ..... count               
base_CircularApertureFlux_6_0_instFluxErr                   ..... count               
base_CircularApertureFlux_9_0_instFlux     

In [14]:
assert(np.all(dia_src_flag)), 'FALSE: not all instFlux entries have units of counts.'
print('All forced_src instFlux entries have units of counts: ',np.all(dia_src_flag))

AssertionError: FALSE: not all instFlux entries have units of counts.